<a href="https://colab.research.google.com/github/Plusholic/Engineer_BigDataAnalysis/blob/main/BigData_practice_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

fatal: destination path 'yemoonsaBigdata' already exists and is not an empty directory.


In [44]:
import pandas as pd
import numpy as np

X_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_X_train.csv")
X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_X_test.csv")
y_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_y_train.csv")

# 확인결과 결측치는 없음. target이 0,1이므로 분류모델이고, 갯수가 bias 되어 있음.
print(X_train.info())
print(X_train)
print(y_train.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education_num   30162 non-null  int64 
 3   marital_status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   capital_gain    30162 non-null  int64 
 9   capital_loss    30162 non-null  int64 
 10  hours_per_week  30162 non-null  int64 
 11  native_country  30162 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.8+ MB
None
       age         workclass  education_num      marital_status  \
0       39         State-gov             13       Never-married   
1       50  Self-emp-not-inc             13  Married-civ-spouse   
2       38         

In [45]:
print(X_train.describe())

                age  education_num  capital_gain  capital_loss  hours_per_week
count  30162.000000   30162.000000  30162.000000  30162.000000    30162.000000
mean      38.437902      10.121312   1092.007858     88.372489       40.931238
std       13.134665       2.549995   7406.346497    404.298370       11.979984
min       17.000000       1.000000      0.000000      0.000000        1.000000
25%       28.000000       9.000000      0.000000      0.000000       40.000000
50%       37.000000      10.000000      0.000000      0.000000       40.000000
75%       47.000000      13.000000      0.000000      0.000000       45.000000
max       90.000000      16.000000  99999.000000   4356.000000       99.000000


In [46]:
# capital_gain, capital_loss 는 bias 되어있는것을 확인할 수 있음. 두 변수의 75 백분위수 ~ 100 백분위수 범위를 5 백분위수 단계로 쪼개서 살펴보기
print(X_train['capital_gain'].quantile([q/20 for q in range(15, 21)])) # quantile(백분위수) -> 해당 데이터의 백분위수에 해당하는 데이터를 반환
print(X_train['capital_loss'].quantile([q/20 for q in range(15, 21)]))

0.75        0.0
0.80        0.0
0.85        0.0
0.90        0.0
0.95     5013.0
1.00    99999.0
Name: capital_gain, dtype: float64
0.75       0.0
0.80       0.0
0.85       0.0
0.90       0.0
0.95       0.0
1.00    4356.0
Name: capital_loss, dtype: float64


In [47]:
X_train['capital_gain_yn'] = np.where(X_train['capital_gain'] > 0, 1, 0) # 0보다 gain이 크면 1, 아니면 0으로 바꿔준다.
X_train['capital_loss_yn'] = np.where(X_train['capital_loss'] > 0, 1, 0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain'] > 0, 1, 0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss'] > 0, 1, 0)

In [48]:
categoric_ = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'capital_gain_yn', 'capital_loss_yn']
numeric_ = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

In [49]:
X_train = X_train.drop([], axis=1)
X_test = X_test.drop([], axis=1)

train_df = pd.concat([X_train, y_train], axis=1)

for _col in numeric_:
  print('-'*80)
  print(_col)
  print(train_df.groupby(['target'])[_col].describe(), end='\n\n')

--------------------------------------------------------------------------------
age
          count      mean        std   min   25%   50%   75%   max
target                                                            
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0

--------------------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       22654.0   9.629116  2.413596  1.0   9.0   9.0  10.0  16.0
1        7508.0  11.606420  2.368423  2.0  10.0  12.0  13.0  16.0

--------------------------------------------------------------------------------
capital_gain
          count         mean           std  min  25%  50%  75%      max
target                                                                 
0       22654.0   148.893838    936.392280  0.0 

In [50]:
# column 별로 target의 평균
for _col in categoric_:
  print(train_df.groupby([_col], as_index=False)['target'].mean().sort_values(by='target', ascending=False), end='\n\n')

          workclass    target
3      Self-emp-inc  0.558659
0       Federal-gov  0.387063
1         Local-gov  0.294630
4  Self-emp-not-inc  0.285714
5         State-gov  0.268960
2           Private  0.218792
6       Without-pay  0.000000

          marital_status    target
1      Married-AF-spouse  0.476190
2     Married-civ-spouse  0.454959
0               Divorced  0.107262
6                Widowed  0.096735
3  Married-spouse-absent  0.083784
5              Separated  0.070288
4          Never-married  0.048324

           occupation    target
3     Exec-managerial  0.485220
9      Prof-specialty  0.448489
10    Protective-serv  0.326087
12       Tech-support  0.304825
11              Sales  0.270647
2        Craft-repair  0.225310
13   Transport-moving  0.202926
0        Adm-clerical  0.133835
6   Machine-op-inspct  0.124619
4     Farming-fishing  0.116279
1        Armed-Forces  0.111111
5   Handlers-cleaners  0.061481
7       Other-service  0.041096
8     Priv-house-serv  0.00699

In [51]:
from sklearn.preprocessing import LabelEncoder
X = pd.concat([X_train, X_test])
for _col in categoric_:
  le = LabelEncoder()
  le.fit(X_train[_col])
  X_train[_col] = le.transform(X_train[_col])
  X_test[_col] = le.transform(X_test[_col])

In [52]:
X_train

,age,workclass,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,capital_gain_yn,capital_loss_yn
0,39,5,13,4,0,1,4,1,2174,0,40,38,1,0
1,50,4,13,2,3,0,4,1,0,0,13,38,0,0
2,38,2,9,0,5,1,4,1,0,0,40,38,0,0
3,53,2,7,2,5,0,2,1,0,0,40,38,0,0
4,28,2,13,2,9,5,2,0,0,0,40,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,2,12,2,12,5,4,0,0,0,38,38,0,0
30158,40,2,9,2,6,0,4,1,0,0,40,38,0,0
30159,58,2,9,6,0,4,4,0,0,0,40,38,0,0
30160,22,2,9,4,0,3,4,1,0,0,20,38,0,0


In [53]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train) # stratify = target 으로 지정해주면 각각의 class의 비율을 train / validation에 유지해줍니다. 한쪽으로 쏠려서 분배되는 것을 방지합니다. # 아주 중요한 옵션

In [54]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[numeric_] = scaler.fit_transform(X_tr[numeric_])
X_val[numeric_] = scaler.fit_transform(X_val[numeric_])
X_test[numeric_] = scaler.fit_transform(X_test[numeric_])

In [55]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

XGBClassifier()

In [56]:
from xgboost import XGBClassifier

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

XGBClassifier()

In [57]:
model_xgb2 = XGBClassifier(n_estimator=1000, learning_rate=0.1, max_depth=10)
model_xgb2.fit(
    X_tr, y_tr.values.ravel(),
    early_stopping_rounds=50,
    eval_metric='auc',
    eval_set=[(X_val, y_val)],
    verbose=10
    )

[0]	validation_0-auc:0.897807
Will train until validation_0-auc hasn't improved in 50 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10]	validation_0-auc:0.907674
[20]	validation_0-auc:0.909063
[30]	validation_0-auc:0.90947
[40]	validation_0-auc:0.910699
[50]	validation_0-auc:0.9113
[60]	validation_0-auc:0.911012
[70]	validation_0-auc:0.910531
[80]	validation_0-auc:0.910101
[90]	validation_0-auc:0.909834
[99]	validation_0-auc:0.909651


XGBClassifier(max_depth=10, n_estimator=1000)

In [58]:
# 모델 성능의 확인을 위해 ROC-AUC 값을 구한다.

from sklearn.metrics import roc_auc_score
y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)
y_pred_xgb2 = model_xgb2.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:,1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:,1])
score_xgb2 = roc_auc_score(y_val, y_pred_xgb2[:,1])

print(score_rf)
print(score_xgb1)
print(score_xgb2)

0.891502912030723
0.913267522684624
0.9113967908734159


In [59]:
pd.DataFrame({'feature' : X_tr.columns,
             'fi_rf' : model_rf.feature_importances_,
             'fi_xgb1' : model_xgb1.feature_importances_,
             'fi_xgb2' : model_xgb2.feature_importances_})

,feature,fi_rf,fi_xgb1,fi_xgb2
0,age,0.222929,0.063376,0.026088
1,workclass,0.049689,0.018222,0.018371
2,education_num,0.134888,0.180339,0.102277
3,marital_status,0.072003,0.135519,0.122725
4,occupation,0.086724,0.034247,0.023529
5,relationship,0.107459,0.292271,0.378091
6,race,0.017098,0.012863,0.016964
7,sex,0.013544,0.034612,0.017868
8,capital_gain,0.107607,0.123087,0.182121
9,capital_loss,0.033621,0.052270,0.069083


In [60]:
# 우리가 라벨링 해 준 capital_gain_yn, loss_yn 등이 라벨링 전 보다 중요도가 낮은 걸 확인할 수 있다. 제거해준다.

X_tr.drop(['capital_gain_yn', 'capital_loss_yn'], axis=1, inplace=True)
X_val.drop(['capital_gain_yn', 'capital_loss_yn'], axis=1, inplace=True)
X_test.drop(['capital_gain_yn', 'capital_loss_yn'], axis=1, inplace=True)

In [61]:
from sklearn.model_selection import GridSearchCV

grid_params = {'n_estimators' : [50, 100, 200],
               'max_depth' : [5, 10, 15],
               'min_samples_split' : [2, 5, 10],
               'min_samples_leaf' : [1,2,4]}

rf_cv = GridSearchCV(estimator=model_rf,
                     param_grid = grid_params,
                     cv=5)
rf_cv.fit(X_train, y_train.values.ravel())
# 실제 실행 시간이 오래 걸리기 때문에 시험에서는 제출이 잘 되지 않을 수 있음. 1분 제한시간이 있기 때문임.
# 너무 오래걸린다면 하이퍼파라미터를 2~3개씩 나눠서 튜닝을 수행하면 됨???

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [41]:
pd.DataFrame(rf_cv.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.428139,0.017027,0.029922,0.000712,5,1,2,50,"{'max_depth': 5, 'min_samples_leaf': 1, 'min_s...",0.842367,0.843196,0.849635,0.851459,0.850630,0.847457,0.003870,63
1,0.809694,0.008964,0.057115,0.001863,5,1,2,100,"{'max_depth': 5, 'min_samples_leaf': 1, 'min_s...",0.841372,0.844853,0.851127,0.850630,0.849801,0.847557,0.003815,60
2,1.721939,0.157311,0.109253,0.000969,5,1,2,200,"{'max_depth': 5, 'min_samples_leaf': 1, 'min_s...",0.840875,0.845848,0.850962,0.849635,0.850133,0.847490,0.003744,61
3,0.408181,0.006688,0.029266,0.000354,5,1,5,50,"{'max_depth': 5, 'min_samples_leaf': 1, 'min_s...",0.839715,0.844025,0.854609,0.852454,0.850796,0.848320,0.005572,55
4,0.802012,0.006941,0.055397,0.000258,5,1,5,100,"{'max_depth': 5, 'min_samples_leaf': 1, 'min_s...",0.841041,0.844522,0.849967,0.848972,0.849635,0.846827,0.003497,76


In [34]:
print(rf_cv.best_params_)

{'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}


In [62]:
model_rf2 = RandomForestClassifier(n_estimators=50,
                                   max_depth=15,
                                   min_samples_leaf=1,
                                   min_samples_split=5)
model_rf2.fit(X_tr,y_tr.values.ravel())

y_pred_rf2 = model_rf2.predict_proba(X_val)
score_rf2 = roc_auc_score(y_val, y_pred_rf2[:,1])
print(score_rf2)

0.913009402852737


In [63]:
grid_params = {'max_depth' : [3, 5, 7, 10],
               'min_child_weight' : [1, 2],
               'colsample_bytree' : [0.6, 0.8],
               'subsample' : [0.6, 0.8]}

xgb_cv = GridSearchCV(estimator = model_xgb1, param_grid=grid_params, cv=5)
xgb_cv.fit(X_tr, y_tr.values.ravel())

print(xgb_cv.best_params_)

{'colsample_bytree': 0.6, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.6}


In [65]:
params = {'max_depth' : 7,
          'min_child_weight' : 1,
          'colsample_bytree' : 0.6,
          'subsample' : 0.6}

model_xgb3 = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_xgb3.set_params(**params)

model_xgb3.fit(X_tr, y_tr,
               early_stopping_rounds=50,
               eval_metric='auc',
               eval_set=[(X_val,y_val)],
               verbose=10)

print(model_xgb3.best_score)

[0]	validation_0-auc:0.862063
Will train until validation_0-auc hasn't improved in 50 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10]	validation_0-auc:0.909251
[20]	validation_0-auc:0.911762
[30]	validation_0-auc:0.913632
[40]	validation_0-auc:0.914352
[50]	validation_0-auc:0.914953
[60]	validation_0-auc:0.915038
[70]	validation_0-auc:0.91595
[80]	validation_0-auc:0.916329
[90]	validation_0-auc:0.916387
[100]	validation_0-auc:0.916531
[110]	validation_0-auc:0.916649
[120]	validation_0-auc:0.916647
[130]	validation_0-auc:0.916554
[140]	validation_0-auc:0.916661
[150]	validation_0-auc:0.916564
[160]	validation_0-auc:0.91657
[170]	validation_0-auc:0.916659
[180]	validation_0-auc:0.916515
[190]	validation_0-auc:0.916309
Stopping. Best iteration:
[145]	validation_0-auc:0.916821

0.916821


In [ ]:
pred = model_xgb3.predict_proba(X_test)[:,1]
pd.DataFrame({'index' : X_test.index, 'target' : pred}).to_csv()